SOLO AÑADIR COSAS DEBAJO DEL CHUNK DE CADA NOMBRE

ESTA PARTE ES PARA JAVIER

ESTA PARTE ES PARA ALMODOVAR

ESTA PARTE ES PARA ALBERTO

In [45]:
# 2 - Average total price per trip including tips

import pandas as pd
data=pd.read_csv("C:/Users/Alberto/Desktop/Máster UC3M/3rd quarter/Computación Escalada/SDComputing_labs/LAB2/tripdata_2017-01.csv",header=0)
print (data.columns)

#We select the columns corresponding to the fare, tip, tolls and extra amount and the improvement surcharge, excluding the taxes.

no_taxes = data['fare_amount'] + data['tip_amount'] + data['tolls_amount'] + data['extra'] + data['improvement_surcharge']

#Now, we compute the mean of this quantity. 
meanprice=no_taxes.mean()
print (meanprice, 'No taxes mean')

#Let's compare this mean value to the mean of the total amount given to the driver.

total = data['total_amount']
total_mean = total.mean()
print (total_mean, 'Total amount mean')

#We can see that there is a difference of close to 0.5$ between these amounts, meaning that the fraction of the total amount spent in taxes
#is close to 0.5$ for every trip. Let's check this fact: 
print(data['mta_tax'].mean())


Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount'],
      dtype='object')
15.577018712603214 No taxes mean
16.078110822888735 Total amount mean
0.4972142408420099


In [35]:
#Now, let's implement the previous computations with pyspark. 
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("prueba").master("local[*]").getOrCreate()

rdd = spark.sparkContext.parallelize(snd_analysis)